In [ ]:
# Installing evaluate lib
!pip install -q evaluate

In [ ]:
# Imports
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import torch
import evaluate
import numpy as np

In [ ]:
model_checkpoint = 'distilbert-base-uncased'

# Define label maps
id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative": 0, "Positive": 1}

# Generate classification model from model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id)

In [ ]:
dataset = load_dataset("shawhin/imdb-truncated")
dataset

In [ ]:
# Create Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

In [ ]:
# Tokenizer function
def tokenize(examples):
    text = examples["text"]

    # tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

In [ ]:
# Add pad token if none exists
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [ ]:
# Tokenize training and validation datasets
tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset

In [ ]:
# Creating data collator
# dynamically pad exaxmples in a given batch tobe as long as longes sequence
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# This is best if you have 400 lines in one batch but only 40 lines in other
# so you dont have to waste resources by adding all those padding

In [ ]:
# Import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

# Defining an evaluation function to pass into trainer later
def compute_metrics(p):
    predictions, lables = p
    predictions = np.argmax(predictions, axis=1)

    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

In [ ]:
peft_config = LoraConfig(task_type="SEQ_CLS", # Sequence classification
                        r=4, # intrinsic rank of trainable weigth matrix
                        lora_alpha=32, # learning rate
                        lora_dropout=0.01, # probablilty of dropout
                        target_modules=['q_lin']) # applying lora to query layer

In [ ]:
model= get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [ ]:
# Hyperparameters
lr = 1e-3
batch_size = 4
num_epochs = 10

In [ ]:
# Defining training arguments

training_args = TrainingArguments(
    output_dir=model_checkpoint + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    eval_strategy="epoch",  # Add this to match save_strategy
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [ ]:
# Create Trainer Object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
# Training the Model
trainer.train()

In [ ]:
# Trained model predictions/performance
model.to('cpu')

print("Trained Model Predictions: ")
for text in text_list:
    inputs = tokenizer.encode(text, return_tensors="pt").to("cpu")

    logits = model(inputs).logits
    predictions = torch.max(logits, 1).indices

    print(text + " - " + id2label[predictions.tolist()[0]])